In [1]:
from google.colab import drive
drive.mount("/content/drive")
import nltk
nltk.download('punkt')
!pip install -U sentence-transformers
from tensorflow.keras.layers import Input, LSTM, Dense, Concatenate
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score
import xgboost as xgb
from tensorflow.keras.models import Model
import numpy as np
import pandas as pd
from nltk.util import ngrams
import re
from pathlib import Path
import string
from sklearn.model_selection import train_test_split
from functools import reduce
from math import log
import itertools
from tqdm import tqdm
import os
import json
from sklearn import svm
from nltk.tokenize.casual import TweetTokenizer
import pickle
from gensim.models.word2vec import Word2Vec
import keras
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.utils import to_categorical, plot_model
from sklearn.metrics import precision_recall_fscore_support
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report
import gensim.downloader as api
import gensim
from sklearn.model_selection import KFold

t = TweetTokenizer()
dir="/content/drive/MyDrive/nlp_project/"
os.chdir(dir)
df=pd.read_csv("train.csv")
output=list(df["is_duplicate"])
output_np=np.array(output)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
output_encode=to_categorical(output_np)
output_encode=np.array(output_encode)

In [3]:
open_file = open("embeddings_para_1", "rb")
embeddings1= pickle.load(open_file)
open_file.close()
open_file = open("embeddings_para_2", "rb")
embeddings2 = pickle.load(open_file)
open_file.close()

In [10]:
np.unique(output_encode)

array([0., 1.], dtype=float32)

In [4]:
embeddings1=np.array(embeddings1)
embeddings2=np.array(embeddings2)
cosines=[]
for i in range(len(embeddings1)):
    val=cosine_similarity(np.vstack((embeddings1[i],embeddings2[i])))[0][1]
    cosines.append(val)
cosines=np.array(cosines)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(cosines, output_np, test_size=0.25, random_state=42)


In [15]:
clf = LogisticRegression(random_state=0).fit(X_train.reshape(-1,1), y_train)
y_pred = clf.predict(X_test.reshape(-1,1))
print("Logisitc Regression(cosine)")
print("Accuracy: ",accuracy_score(y_test, y_pred)*100)
print("Classification report:\n", classification_report(y_test, y_pred))
print("F1 Score:\n ",f1_score(y_test, y_pred)*100)

Logisitc Regression(cosine)
Accuracy:  73.15678573901688
Classification report:
               precision    recall  f1-score   support

           0       0.78      0.80      0.79     64078
           1       0.64      0.62      0.63     37009

    accuracy                           0.73    101087
   macro avg       0.71      0.71      0.71    101087
weighted avg       0.73      0.73      0.73    101087

F1 Score:
  62.66767558643461


In [16]:
xgb_cl = xgb.XGBClassifier()
xgb_cl.fit(X_train.reshape(-1,1), y_train)
preds = xgb_cl.predict(X_test.reshape(-1,1))
print("XGBoost(cosine)")
print("Accuracy: ",accuracy_score(y_test, y_pred)*100)
print("Classification report:\n", classification_report(y_test, y_pred))
print("F1 Score:\n ",f1_score(y_test, y_pred)*100)

XGBoost(cosine)
Accuracy:  73.15678573901688
Classification report:
               precision    recall  f1-score   support

           0       0.78      0.80      0.79     64078
           1       0.64      0.62      0.63     37009

    accuracy                           0.73    101087
   macro avg       0.71      0.71      0.71    101087
weighted avg       0.73      0.73      0.73    101087

F1 Score:
  62.66767558643461


In [13]:
input=np.hstack((embeddings1,embeddings2))

In [14]:
from sklearn.manifold import TSNE
tsne1 = TSNE(random_state = 0, n_components=2,n_iter=3000).fit_transform(input)
plt.scatter(tsne1[:,0],tsne1[:,1],s=10,c=output_np,label=output_np)
plt.legend()
plt.show()

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


KeyboardInterrupt: ignored

In [4]:
pca = PCA(n_components=10)
principalComponents = pca.fit_transform()
X_train, X_test, y_train, y_test = train_test_split(input, output_np, test_size=0.25, random_state=42)
clf = LogisticRegression(random_state=0,max_iter=200,tol=0.01).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print("Logisitc Regression(sbert_embedding): ",accuracy_score(y_test, y_pred)*100)
xgb_cl = xgb.XGBClassifier(max_iter=1000,tol=0.01)
xgb_cl.fit(X_train, y_train)
preds = xgb_cl.predict(X_test)
print("XGBoost(sbert_embedding): ",accuracy_score(y_test, preds)*100)

KeyboardInterrupt: ignored